# 1.Setup

In [1]:
#%pip install ultralytics
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
print(os.getenv("KMP_DUPLICATE_LIB_OK"))

TRUE


In [262]:
import ultralytics
from ultralytics import YOLO
ultralytics.checks()
import torch
import torchvision.transforms as T
import numpy as np
import yaml
from pathlib import Path
from scipy.optimize import minimize
import json
import joblib
from sklearn.svm import SVR  # Support Vector Regression
from PIL import Image
import cv2

torch.cuda.is_available()

Ultralytics 8.3.49  Python-3.8.20 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
Setup complete  (32 CPUs, 127.7 GB RAM, 929.1/950.9 GB disk)


True

### Download dataset

In [ ]:
#!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="******************")
project = rf.workspace("********").project("udw") # Check the data.yaml file to get the download link
version = project.version(8)
dataset = version.download("yolov11")
                

# 2. Train YOLOv11

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolo11n.yaml')  # build a new model from scratch
model = YOLO('yolo11n.pt')  # load a pretrained model (recommended for training)

# Use the model
results = model.train(data="../UDW-8/data.yaml", epochs=500, workers=0 )  # train the model 
results = model.val()  # evaluate model performance on the validation set
#results = model('https://ultralytics.com/images/bus.jpg')  # predict on an image
#results = model.export(format='onnx')  # export the model to ONNX format

# 3. RESUME TRAINING

In [ ]:
from ultralytics import YOLO

# Load the partially trained model
model = YOLO("..runs\\detect\\train6\\weights\\last.pt")

# Resume training
results = model.train(resume=True)

# 4. SVM MODEL TRAINING

In [37]:
# Load configurations from YAML files
with open('..UDW-8\\data.yaml', 'r') as f:
    data_config = yaml.safe_load(f)

In [39]:
# Dataset paths
train_path = Path(data_config['train'])
val_path = Path(data_config['val'])
test_path = Path(data_config['test'])


In [40]:
# SVM model
svm_model = SVR(kernel="rbf", C=1.0, epsilon=0.1)  # Initialize SVM regressor
#Load Yolo model
model = YOLO("Yolov11 custom model/weights/best.pt")

In [41]:
def load_ground_truth_from_labels(label_path):
    """
    Loads ground truth bounding boxes for a given image from the labels directory.
    Assumes YOLO format annotations are stored in .txt files with the same name as the image.
    """
    ground_truth_boxes = []
    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5:
                continue  # Skip invalid lines
            _, x_center, y_center, width, height = map(float, parts)
            ground_truth_boxes.append([x_center, y_center, width, height])

    return ground_truth_boxes

In [42]:
def yolo_to_xyxy(box, img_width, img_height):
    """
    Convert YOLO format bounding box to xyxy format.
    Args:
    - box: [x_center, y_center, width, height] in YOLO normalized format.
    - img_width, img_height: Dimensions of the image.
    
    Returns:
    - xyxy: [x_min, y_min, x_max, y_max].
    """
    x_center, y_center, width, height = box
    x_min = (x_center - width / 2) * img_width
    y_min = (y_center - height / 2) * img_height
    x_max = (x_center + width / 2) * img_width
    y_max = (y_center + height / 2) * img_height
    return [x_min, y_min, x_max, y_max]

def compute_iou(box1, box2):
    """
    Compute IoU between two bounding boxes in xyxy format.
    Args:
    - box1, box2: [x_min, y_min, x_max, y_max].

    Returns:
    - IoU value as a float.
    """
    x_min = max(box1[0], box2[0])
    y_min = max(box1[1], box2[1])
    x_max = min(box1[2], box2[2])
    y_max = min(box1[3], box2[3])

    intersection = max(0, x_max - x_min) * max(0, y_max - y_min)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])

    union = area1 + area2 - intersection
    return intersection / union if union > 0 else 0

In [136]:
def calculate_iou_for_image(image_path ,yolo_model):
    """
    Calculate IoU for each detected object against the ground truth.
    Args:
    - image_path: Path to the image.
    - yaml_file: Path to the dataset YAML file.
    - yolo_model: YOLO model object.

    Returns:
    - iou_scores: List of IoU scores for each prediction.
    """
    # Load ground truth bounding boxes
    # Extract the file name without extension
    dir_path, file_name = os.path.split(image_path)
    file_name_without_ext, ext = os.path.splitext(file_name)

    # Replace 'images' with 'label' and append the '.txt' extension
    label_path = os.path.join(dir_path.replace("images", "labels"), file_name_without_ext + ".txt")
    
    if not os.path.exists(label_path):
        raise ValueError(f"Annotation file {label_path} not found.")
    
    ground_truth_boxes = load_ground_truth_from_labels(label_path)

    # Load image and run detection
    results = yolo_model.predict(image_path)
    predictions = results[0].boxes  # Predicted boxes

    # Load image dimensions
    image = results[0].orig_img
    img_height, img_width = image.shape[:2]

    # Convert ground truth boxes to xyxy format
    ground_truth_xyxy = [yolo_to_xyxy(box, img_width, img_height) for box in ground_truth_boxes]

    # Extract predictions in xyxy format
    predicted_boxes = predictions.xyxy.cpu().numpy()

    # Compute IoU scores
    iou_scores = []
    for pred_box in predicted_boxes:
        iou_with_all_gt = [compute_iou(pred_box[:4], gt_box) for gt_box in ground_truth_xyxy]
        best_iou = max(iou_with_all_gt) if iou_with_all_gt else 0
        iou_scores.append(best_iou)

    return iou_scores

In [174]:
def get_yolo_outputs(image,image_path):
    
    """Runs YOLOv11 on an image and returns outputs."""
    results = model.predict(image)
    # print(type(results))
    # Extract confidence and IoU scores (adapt based on YOLOv11 output structure)
    if hasattr(results[0].boxes, 'xyxy') and len(results[0].boxes.xyxy) > 0:
        confidences = results[0].boxes.conf.cpu().numpy()  # Confidence scores results[0].boxes.id.int().cpu().tolist()
        # print(confidences)
        iou_score = np.array(calculate_iou_for_image(image_path , model) )   # IoU scores
        return confidences.mean(), iou_score.mean()
    else:
        raise ValueError("Invalid YOLO output structure. Please verify model compatibility.")

In [175]:
# Data augmentation parameters to predict
PARAMETERS = ['brightness', 'gamma', 'contrast', 'hue', 'invert', 'solarize', 'posterize', 'gaussian_blur', 'saturation', 'sharpness']

# Apply transformations based on parameters
def apply_transformations(image, params):
    transform_list = []
    
    transform_list.append(T.ColorJitter(brightness=params[0], contrast=params[2], saturation=params[8], hue=params[3]))
    if params[4]:  # Invert
        transform_list.append(T.RandomInvert())
    transform_list.append(T.RandomSolarize(params[5]))
    transform_list.append(T.RandomPosterize(bits=int(params[6])))
    transform_list.append(T.GaussianBlur(kernel_size=(5, 5), sigma=params[7]))
    transform_list.append(T.RandomAdjustSharpness(sharpness_factor=params[9]))

    transform = T.Compose(transform_list)
    return transform(image)


In [176]:
# Self-supervised optimization objective
def optimization_objective(params, image,img_path):
    transformed_image = apply_transformations(image, params)
    conf, iou = get_yolo_outputs(transformed_image,img_path)
    # Maximize confidence and IoU, minimize negative sum
    return -(conf + iou)

In [177]:
# Optimize transformation parameters
def optimize_parameters(image,img_path):
    initial_params = np.random.rand(len(PARAMETERS))  # Random initialization
    bounds = [(0, 1), (0.5, 2), (0, 1), (-0.5, 0.5), (0, 1), (0, 255), (1, 8), (0, 5), (0, 2), (0, 2)]

    result = minimize(optimization_objective, initial_params, args=(image,img_path), bounds=bounds, method='L-BFGS-B')
    return result.x


In [317]:
def batch_processing(image_paths, epoch, batch_size):
    features = []
    targets = []  # Targets are YOLO confidences + IoU scores
    for i in range(0, len(image_paths), batch_size):
        batch = image_paths[i:i+batch_size]

        for img_path in batch:
            try:
                # Debugging log
                print(f"Processing image: {img_path}")

                # Load image as float
                image = cv2.imread(img_path, cv2.IMREAD_COLOR).astype('float32')
                image = torch.from_numpy(image)
                image = image.unsqueeze(0)  # Adds a batch dimension
                image = image.permute(0, 3, 1, 2)
                #image = T.ToTensor()(Image.open(img_path))
                #image = image.type(torch.float32) / 128.0 - 1.0 # [-1, 1]
                #image = (image + 1.0) * 128.0  # case [-1, 1]
                image = image.type(torch.float32) / 255.0  # [0, 1]
                #image = image * 255.0  # case [0, 1]
                #image = torch.clip(image, 0.0, 255.0)
                #image = image.type(torch.uint8)

                
                # Debugging log for YOLO outputs
                conf, iou = get_yolo_outputs(image,img_path)
                print(f"YOLO Output - Confidence: {conf}, IoU: {iou}")
            
                # Optimize parameters for each image
                optimized_params = optimize_parameters(image,img_path)

                # Debugging log for optimized parameters
                print(f"Optimized Parameters: {optimized_params}")

                # Apply optimized transformations
                transformed_image = apply_transformations(image, optimized_params)

                # Recheck YOLO outputs after transformations
                new_conf, new_iou = get_yolo_outputs(transformed_image,img_path)
                print(f"Transformed YOLO Output - Confidence: {new_conf}, IoU: {new_iou}")

                # Collect features and targets for SVM training
                features.append(optimized_params.tolist())
                targets.append(new_conf + new_iou)

            except Exception as e:
                print(f"Error processing image {img_path}: {e}")
                continue

    # Safeguard: Check if features and targets are not empty
    if not features or not targets:
        raise ValueError(f"No valid data generated in epoch {epoch}. Please check the input images and YOLO model.")

    print(f"Epoch {epoch}: Processed {len(features)} images.")
    return np.array(features), np.array(targets)

In [318]:
def train_svm_model(features, targets):
    """Train SVM using YOLO features (confidence, IoU) and transformation parameters as input."""
    # Reshape features to ensure 2D array format
    if features.ndim == 1:
        features = features.reshape(-1, 1)
    if targets.ndim == 1:
        targets = targets.reshape(-1)

    svm_model.fit(features, targets)
    print("SVM model trained successfully.")
    return svm_model

In [319]:
# Save results to JSON files
def save_results(results, filename):
    with open(filename, 'w') as f:
        json.dump(results, f, indent=4)

# Save SVM model
def save_model(model, filename):
    joblib.dump(model, filename)
    print(f"SVM model saved to {filename}")

In [320]:
def main_pipeline(epochs, batch_size):
    # Load training, validation, and test images from data.yaml paths
    train_images = list(Path(data_config['train']).glob("*.jpg"))
    val_images = list(Path(data_config['val']).glob("*.jpg"))
    test_images = list(Path(data_config['test']).glob("*.jpg"))
    print(len(train_images))
    # Training Loop with Epochs
    for epoch in range(epochs):
        train_features, train_targets = batch_processing(train_images, epoch,batch_size)
        svm_model = train_svm_model(train_features, train_targets)

    # Save the trained SVM model after training is complete
    save_model(svm_model, 'trained_svm_model.joblib')

    # Validation phase (only save results)
    val_features, val_targets = batch_processing(val_images, epoch="Validation")
    save_results({"features": val_features.tolist(), "targets": val_targets.tolist()}, "validation_results.json")

    # Testing phase (only save results)
    test_features, test_targets = batch_processing(test_images, epoch="Testing")
    save_results({"features": test_features.tolist(), "targets": test_targets.tolist()}, "testing_results.json")

    return {
        "train": {"features": train_features, "targets": train_targets},
        "val": {"features": val_features, "targets": val_targets},
        "test": {"features": test_features, "targets": test_targets},
    }

In [ ]:
results = main_pipeline(10,12)
print("Pipeline executed successfully. Results and model saved.")

In [2]:
!chmod u+rwx /workspaces/Yolo_Model_Enhancement/train6.zip